### 4. Steuerung des Minings

Das Mining eines *FullNodes* sollte parallel zu allen anderen Prozessen des Nodes ablaufen. Ansonsten kann der *FullNode* nicht schnell genug auf neue Informationen reagieren und das Mining wäre folglich ineffizient. Die Paralellisierung von Prozessen in Python kann mit Hilfe des `threading` Moduls erfolgen. Da die Implementierung von Parallelisierung in Python nicht in Zentrum unserer Vorlesung steht, sind die Erläuterungen in diesem Notebook etwas weniger detailliert im Vergleich zu den vorangeganenen Notebooks. Ein Tutorial zu `Thread`s in Python gibt es [hier](https://pymotw.com/3/threading/).  
Bisher laufen in unserem Python-Programm alle Prozesse sequeniell ab. D.h., wenn z.B. kurz nach dem Posten einer Transaktion an einen *FullNode* auch noch ein neuer Block gepostet wird, so wird vom *FullNode* zuerst die komplette Prozesskette für die Transaktionsverarbeitung durchlaufen, bevor anschließend der Block verarbeitet wird. Im Rahmen unseres Prototyps enstehen dadurch bei den bisher eingeführten Prozessen im Zusammenhang mit *Flask* nur geringe Effizienzverluste, die wir ohne weiter darüber nachzudenken vernachlässigt haben. Da es aus Mining-Perspektive jedoch erhebliche Nachteile mit sich bringt, wenn der Miner nicht an der Blockchain mined, die von der Mehrheit der *FullNodes* im Netzwerk als *Primary-Chain* eingestuft wird, ist die Parallelisierung des Minings mehr oder weniger notwendig.  
Um das Mining des *FullNodes* parallel zu allen anderen Prozessen ablaufen zu lassen, gleichzeitig aber zu gewährleisten, dass innerhalb des Miningprozesses auf evtl. Aktionen der anderen Prozesse reagiert werden kann, verwenden wir das im vorangeganen Notebook eingeführte `PAUSE_MINING_EVENT`. Der Status dieses Events wird innerhalb der `mine_block()` Methode der `FullNode` Klasse ständig abgeprüft und das Minen wird abgebrochen, sobald das Event getriggert wird. Das Event wird von uns z.B. getriggert, wenn die *Primary-Chain* des Full Nodes verändert wurde.  
Die Implementierung des Parallelprozesses erfolgt mit Hilfe der neuen Klasse `MiningLoopThread`, die von der `Thread` Klasse erbt (deshalb bei der Klassendefinition das `(Thread)` im Code). Wir fügen die Klasse der `run_full_node.py` zu Beginn hinzu. Wichtig ist, dass wir bei Initialisierung einer `MiningLoopThread`-Instanz alle Objekte übergeben, die zwischen Mainlineprozessen (damit bezichnen wir von jetzt an alle Nicht-Mining-Prozesse, die in einem *FullNode* ablaufen) und Parallelprozess geteilt werden sollen. Dies geschieht im Konstruktor der Klasse `MiningLoopThread`. Am Ende des Konstruktors der `MiningLoopThread`-Klasse wird der Konstruktor der `Thread` Klasse aufgerufen. Die eigentliche Implementierung des Parallelprozesses erfolgt dann in der `run` Methode der `MiningLoopThread`-Klasse. Wir steuern das Mining über eine unendliche Loop (`while True:`). Gemined wird nur, wenn der *Mempool* der *Primary-Chain* nicht leer ist und das `self.pause_mining_event` Event nicht getriggert ist. Ansonten wird eine kurze Pause eingelegt (5 Sekunden, willkürlich festgelegt), um dann nochmal den Status des *Mempools* und des *Events* zu übeprüfen um mit dem Mining eventuell dann zu beginnen. Wichtig ist, dass bei Aufruf der `mine_block`-Methode das `self.pause_mining_event`(wie auch bei der `ìf`-Anweisung zuvor) übergeben wird und nicht das `pause_mining_event`. Da Python mit Pointern arbeitet, zeigen die Variablen zwar beide auf den gleichen Speichereintrag, der Parallelprozess (repräsentiert durch die `MiningLoopThread` Klasse in unserem Fall) kann aber nur mit *eigenen* Variablen arbeiten.



```python
## New (4_5) run 1 -- (THIS ENTIRE CLASS IS NEW)
class MiningLoopThread(Thread):
    def __init__(self, pause_mining_event, full_node):
        # Creates a MiningLoopThread instance which inherits
        # from the Thread class. It is important to note,
        # that all external parameters that should also be
        # known by this Thread have to be assigned to this
        # Thread as well. I.e., in our case this is the pause_mining_event
        # and the full_node.
        self.pause_mining_event = pause_mining_event
        self.full_node = full_node

        # After the parameters are passed to the Thread
        # it calls the constructor of the parent class.
        Thread.__init__(self)

    def run(self):
        # This defines (or runs) the process we want to run parallel,
        # i.e., the Mining loop.

        # We run this loop forever
        while True:
            # Start mining only if the mempool of the
            # full_node's current primary chain is not empty
            # and if pause_mining_event is not set.
            if self.full_node.get_primary_chain().mempool and not self.pause_mining_event.is_set():
                print('Start mining Block.')

                # This triggers the actual mining.
                # Note that we slightly modify the mine_block() method
                # from the `FullNode` class as shown below to take 
                # care of the selfe.pause_mining_event.
                mined_block = self.full_node.get_primary_chain().\
                mine_block(self.full_node.public_key_receiver_coinbase_transaction,\
                self.pause_mining_event)
                if mined_block is not None:

                    # Add the mined block to the primary chain
                    # and also add its id to the list of known ids.
                    backup_chain = copy.deepcopy(self.full_node.get_primary_chain())
                    test_dummy = self.full_node.get_primary_chain().add_block(mined_block)
                    if test_dummy:
                       self.full_node.add_backuped_chain(backup_chain)

                    self.full_node.seen_block_ids.append(mined_block.id)


                    # This is more or less debugging output:
                    print('Successfully mined block with id ' \
                    + str(mined_block.id[:8]) + '...')
                    print('Current prim. chain length ' + \
                    str(len(self.full_node.get_primary_chain().chain)) + '.')
                    print('Current mempool size ' +\
                    str(len(self.full_node.get_primary_chain().mempool.keys()))'.')

                    # Propagate the block to neighbors
                    self.full_node.propagate_block(mined_block.id)

            # Note that the event is cleared in the process_incoming_block 
            # routine when processing is finished.

            # This is entered if pause_mining_event is set.
            elif self.pause_mining_event.is_set():
                print('Primary chain changed, mining paused.')
                # Sleep is not necessary but it's easier this way, i.e. otherwise
                # mining might restart to quickly then stop again etc.
                time.sleep(5)

            # This is entered if the primary chain's mempool is empty
            else:
                self.full_node.logger.info('Nothing to mine at the moment, mining paused.')
                # Sleep is not necessary but it's easier this way
                time.sleep(5)


```

Zusätzlich modifizieren wir die `mine_block()` Methode der `Blockchain` Klasse minimal, weil durch den oben implementierten Aufruf der Methode zusätzlich zum bereits implementierten `public_key_coinbase_transaction` nun auch noch das `pause_mining_event` übergeben wird und wir mit Hilfe dieses Events erreichen wollen, dass nur solange gemined wird, bis das Event getriggert wurde.

```python
 ## New(4_5) 1
    def mine_block(self, public_key_receiver_coinbase_transaction,\
    pause_mining_event):
...

            ## New(4_5) 2
            while not candidate_block_id[:self.initial_difficulty]\
            == '0'*self.initial_difficulty \
            and not pause_mining_event.is_set():

...
            ## New(4_5) 3
            if pause_mining_event.is_set():
                # Sleep not necessary/efficient 
                # but easier that way to test
                sleep(1)
                return(None)
```

* *# New(4_5) 1*: pause_mining_event wird als zusätzlicher Parameter überbeben
* *# New(4_5) 3*: Gemined wird nur solange bis das Event getriggert wird
* *# New(4_5) 3*: Wenn das Event getriggert wurde, wird die methode an dieser Stelle verlassen.


Um den Parallelprozess im Code zu starten, muss zuerst eine Instanz der `MiningLoopThread` Klasse erzeugt werden und anschließend die `run` Methode dieser Instanz aufgerufen werden. Die entsprechenden Stellen sind in der Datei `run_full_node.py` gekennzeichnet:

```python
...
## New (4_5) run 2
# Initialize MiningLoop
minig_loop_thread = MiningLoopThread(PAUSE_MINING_EVENT,full_node)
...
    ## New (4_5) run 3
    # Start the mining loop
    minig_loop_thread.start()
...
```

Wenn das `FullNode` Mining über Flask gesteuert wird, muss außerdem noch eine Möglichkeit für die dynamische *Public Key* Übergabe geschaffen werden. Jeder *FullNode* möchte den *Mining-Reward* schließlich an eine eigene Adresse überweisen und wir wollen hierfür nicht immer den Programmcode verändern (das wäre zwar möglich, aber umständlich). Weil der *Public Key* auf Grund seiner Länge nur umständlich über die Kommandozeile direkt übergeben werden kann, implementieren wir dies indem bei Aufruf der `run_full_node.py` zusätzlich zu den Parametern `-port` und `-neighbors` noch ein Parameter `-public_key_file_name` übergeben wird. Durch `public_key_file_name` wird eine Datei (im einfachsten Fall im gleichen Verzeichnis) referenziert, die den *Public-Key* enthält, an den der *Mining-Reward* des *FullNodes* überwiesen werden soll. Selbstverständlich muss die entsprechende Datei auch vorhanden sein. In der Datei steht nur der *Public-Key* im Hexadezimalformat, ansonsten nichts, im Idealfall auch keine Zeilenumbrüche oder Whitespaces etc. Wir setzten die Defaulteinstellungen für die Übergabe des `-public_key_file_name` Parameters so, dass für den Fall der Nichtübergabe des Parameters, automatisch die Datei `public_key.txt` verwendet wird.

```python
...

    ## New (4_5) run 4
    parser.add_argument('-pk','--public_key_file_name', default='public_key.txt',\
    type=str, help='Name of the file containing the public key for the coinbase transactions of the node.')
...

    ## New (4_5) run 5
    public_key_file_name = args.public_key_file_name

...

    ## New (4_5) run 6
    # This opens the public_key_file_name file and reads
    # the public key. then the key is passed to the full_node
    with open(public_key_file_name, 'r') as key_file:
        # Just in case it was pasted with linebreaks
        full_node.public_key_receiver_coinbase_transaction=key_file.read()
```

#### Test von Blockkommunikation, Initialisierung und Mining

Um die Implementierungen bzgl. des Blockaustauschs (4_4), der *FullNode* Initialisierung (4_4) und des Minings (4_5) zu testen, gehe folgedermaßen vor: Erstelle in diesem Verzeichnis zuerst drei `public_key_xyz.txt` Dateien. Die erste benennst Du `public_key_gen.txt` und speicherst den *Public Key*  der Genesis Wallet darin ab.   

`3059301306072a8648ce3d020106082a8648ce3d030107034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e`

Die beiden anderen Dateien nennst Du `public_key_a.txt` und `public_key_b.txt` und speicherst darin jeweils den *Public Key* A:

`3059301306072a8648ce3d020106082a8648ce3d03010703420004a557c098f9fd58388690eb4633bc81f2a4e2ae341f2b81dbaef27f8aef686fb793632412ad7d557d8f5d57077d59b9491997cbf4480dabf980f675a29a3ce370`

und den *Public Key* B:

`3059301306072a8648ce3d020106082a8648ce3d03010703420004d4eb72cc94978f09f96d981214b10da0d06c75797acada7dd844ace222026330e7817f0cb6088ed7000f916e01ba3497c696177b72a1ddfbbbd1db6914410389`

*Hinweis:* Um die folgenden Schritte ausführen zu können, benötigst Du das Modul `numpy` im `fed_coin` Environment. Aktiviere das Environment zuerst (Windows: `activate fed_coin` Mac: `source activate fed_coin`). Installiere dann das `numpy` Modul über folgenden Befehl:

`pip install numpy`

Anschließend startest Du zunächst einen *FullNode Genesis* mit dem folgendne Befehl in der Kommandozeile. Achte darauf, dass Du mit der Kommandozeile im aktuellen Verzeichiss bist, dass das `fed_coin` Environment aktiv ist und enventuelle andere *FullNodes* vorher beendet wurden.


`python run_full_node.py -p 5000 -pk public_key_gen.txt`

#####  1. Mining
Nun erzeugst Du zwei Wallets (Genesis und A) und sendest eine Transaktion von der Genesis Wallet an die Wallet A (s. u.). Im Anschluss daran sollte das Mining am *FullNode Genesis* starten. Du solltest eine Info darüber in der Kommandozeile erhalten. Warte bis der erste Block fertig gemined wurde (das dauert etwas) und mache dann mit den Tests bzgl. der Initialisierung unten weiter.

In [ ]:
# Automatically relaoad modules that have changed while
# being loaded.
%reload_ext autoreload
# Set equal to 2 to enable auto realod,
# set equal to 0 to disable auto relaod.
%autoreload 2

In [ ]:
# Import stuff
# Import the modules we need
from IPython.core.debugger import set_trace

from FEDCoin import *
from utils import *

import numpy as np
import time
import requests

In [ ]:
# Testing it:
# Note that we use a similar code as in 4_2 for this step.

gen_priv_key = '308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'
gen_address = '127.0.0.1:5000'
a_address = '127.0.0.1:5001'
gen_wallet = Wallet(gen_priv_key, gen_address)

# Check the balance (should output 50 if the ledger is in genesis state):
print(gen_wallet.balance)
## Same as 4_2_Teil_2 --end##

# Creste a new receiver wallet (more or less random, doesn't
# have to be connected)
a_priv_key = '308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042075189dae1d094c49143d0773fd8860bab4d012112e3da1e9628bd9c0f4d50befa14403420004a557c098f9fd58388690eb4633bc81f2a4e2ae341f2b81dbaef27f8aef686fb793632412ad7d557d8f5d57077d59b9491997cbf4480dabf980f675a29a3ce370'
a_wallet = Wallet(a_priv_key)

# Create signed Transaction and post it.
signed_transaction = gen_wallet.create_signed_transaction(a_wallet.public_key, 2)
gen_wallet.post_signed_transaction(signed_transaction)

##### 2. Initialisierung (1)
Starte im Anschluss daran einen zweiten *FullNode A* mit der foldender Eingabe in einer neuen Kommandozeile (Verzeichniss muss stimmen und `fed_coin` Environment muss aktiv sein)':

`python run_full_node.py -p 5001 -n 127.0.0.1:5000 -pk public_key_a.txt`

Der *FullNode A* sollte nun basierend auf dem *FullNode Genesis* initialisiert werden. Wenn die Ausgabe in der Kommandozeile das bestätigt, hast Du alles richtig gemacht - die Initialisierung eines *FullNodes* scheint zu funktionieren.  

##### 3. Blockkommunikation
Nun ist das Ziel die Kommunikation der *FullNodes* bzgl. der Blöcke und der Integration der damit verbundenen Prozesse in das Mining zu testen. Generiere hierfür einen ständigen Strom von Transaktionen und sende eine signierte Transaktion jeweils an einen aktiven Knoten im Netzwerk (*FullNode Genesis* oder *FullNode A*). Du kannst versuchen möglichst viele gültige Transaktionen zu erzeugen, damit der Test flüssig abläuft. Beachte bei der Transaktionserzeugung, dass die `balance` einer *Wallet* immer dem aktuellen Stand der *Primary Chain* des entsprechenden *Parent Nodes* der *Wallet* entspricht. Transaktionen die noch im Mempool hängen, werden dabei nicht berücksichtigt.  
Übeprüfe anhand der Ausgabe in den Kommandozeilen der beiden *FullNodes* ob die Kommunikation und das Mining wie gewünscht abläuft. Wenn Du denkst, dass alles nach Plan läuft gehe zum letzten Test über. Wichtig für den letzten Test ist, dass die beiden *FullNodes* aus diesem Test nicht beendet werden (bzw. zumindest der *FullNode Genesis* sollte weiterlaufen).

In [ ]:
# Testing it:
a_address = '127.0.0.1:5001'

# Reinitialize the a_wallet since the FullNode A is running now
a_wallet = Wallet(a_priv_key,a_address)


list_of_wallets = [gen_wallet, a_wallet]


# Run this until it is stopped manually
while True:
    # Choose a random wallet out of the wallet list
    sending_wallet = np.random.choice(list_of_wallets)
    sending_wallet.calculate_balance()
    
    # Check if the wallet has a balance of at least 
    # 1 (not necessary but more efficient)
    while sending_wallet.balance < 1:
        print('Account balance too low, waiting for 5 seconds.')
        # Wait for a little bit
        time.sleep(5)
        # Choose a new wallet (possibly the same)
        sending_wallet = np.random.choice(list_of_wallets)
        sending_wallet.calculate_balance()
    
    # Send small amounts only, because mempool transactions 
    # are not regarded in balance calculation
    possible_amounts = [i for i in range(1,int(np.floor(min(5,sending_wallet.balance)))+1)]
    sending_amount = float(np.random.choice(possible_amounts))
    print('Amount: ' + str(sending_amount))
    
    # The recieving wallet simply has to be another wallet (this is not a formal requirement)
    receiving_wallet = np.random.choice([i for i in list_of_wallets if i!=sending_wallet])
    
    # Create a transaction and post it
    transaction = sending_wallet.create_signed_transaction(receiving_wallet.public_key,sending_amount)
    sending_wallet.post_signed_transaction(transaction)

    time_to_sleep = np.random.choice([i for i in range(1,21)])
    print('Transaction sent, sleeping for ' + str(time_to_sleep) + ' seconds.')

    time.sleep(time_to_sleep)

##### 4. Initialisierung (2)

Starte im Anschluss daran noch einen dritten *FullNode B* mit der foldender Eingabe in einer neuen Kommandozeile (Verzeichnis muss stimmen und `fed_coin` Environment muss aktiv sein)':

`python run_full_node.py -p 5002 -n 127.0.0.1:5000 -pk public_key_b.txt`

Die Ausgabe in der Kommandozeile sollte jetzt einer *FullNode* Initialisierung mit mehreren Blöcken entsprechen.